#### This code is used to construct variables used in baseline regression
- open B-A (x): dummy difference (the state is open to interstate banking in the year) between target B and acquirer A
- Acq. vol. A buys B (y1): total transaction value of acquisitions made by firms located in A targeting firms in B/total transaction value of acquisition made by firms located in A
- Acq. vol. A buys B (y2): the number of transactions made by firms located in A targeting firms in B/the number of transactions made by firms located in A
- Within-state acquisition growth 
- Tobin’s q B-A
- Stock return B-A
- GDP growth B-A
- GDP per capita B-A
- Unemployment B-A
- Corporate tax B-A
- Anti-combination B-A
- Industry dissimilarity A&B
- Economic correlation A&B

In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',1000)

In [2]:
## Filter samples
ma_path = '../M&A1980_1997/'
ma_df = pd.DataFrame()
for f in os.listdir(ma_path):
    df = pd.read_excel(os.path.join(ma_path,f),header = 1)
    df = df.iloc[:-10,:]
    ma_df = pd.concat([ma_df,df])

### exclude finance and utility industry and other exceptions
ma_df = ma_df[~ma_df['Target\nPrimary\n  SIC\n Code'].isin(range(6000,7000))]
ma_df = ma_df[~ma_df['Target\nPrimary\n  SIC\n Code'].isin(range(4900,5000))]
ma_df = ma_df[~ma_df['Acquiror\nPrimary\n  SIC\n  Code'].isin(range(6000,7000))]
ma_df = ma_df[~ma_df['Acquiror\nPrimary\n  SIC\n  Code'].isin(range(4900,5000))]

ma_df = ma_df[~ma_df['Target\nPrimary\n  SIC\n Code'].isin(['619A','999F','499A','999G','619B','999A','999B','999C','999D','999E'])]
ma_df = ma_df[~ma_df['Acquiror\nPrimary\n  SIC\n  Code'].isin(['619A','999F','499A','999G','619B','999A','999B','999C','999D','999E'])]

ma_df['Target\nPrimary\n  SIC\n Code'] = ma_df['Target\nPrimary\n  SIC\n Code'].astype(int)
ma_df['Acquiror\nPrimary\n  SIC\n  Code'] = ma_df['Acquiror\nPrimary\n  SIC\n  Code'].astype(int)
print('There are only ',len(ma_df),' left.')
## exclude state as 'Unknown','Foreign'
ma_df = ma_df[(~ma_df['Acquiror State'].isin(['Unknown','Foreign','Virgin Islands','Cayman Islands']))&(~ma_df['Target State'].isin(['Unknown','Foreign','Virgin Islands','Cayman Islands']))]
ma_df.dropna(subset=['Target State','Acquiror State'],how='any',inplace=True)
print('There are only ',len(ma_df),' left.')

ma_df['announce_year'] = ma_df['  Date\nAnnounced'].apply(lambda x:x.year)

## State and state abbr
state_abbreviations = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "D. of Columbia":"DC",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY"
}

state_full = {}
for k,v in state_abbreviations.items():
    state_full[v] = k

There are only  64377  left.
There are only  53403  left.


In [3]:
## before construct measures, we firstly replicate Table 1
ma_df['same_state'] = ma_df.apply(lambda row: 1 if row['Acquiror State']==row['Target State'] else 0, axis=1)
acquire_grouped = ma_df.groupby(by = ['Acquiror State','same_state'])
acquire_count = acquire_grouped['Target Name'].count()
acquire_count = acquire_count.unstack(level=-1)
acquire_count.columns = ['NC','NI']

acquire_volume = acquire_grouped['Value of\nTransaction\n($mil)'].sum(numeric_only = True)
acquire_volume = acquire_volume.unstack(level=-1)
acquire_volume.columns = ['VC','VI']

target_grouped = ma_df.groupby(by = ['Target State','same_state'])
target_count =target_grouped['Acquiror Name'].count()
target_count = target_count.unstack(level=-1)
target_count.columns = ['NC','NI']

target_volume = target_grouped['Value of\nTransaction\n($mil)'].sum(numeric_only = True)
target_volume = target_volume.unstack(level=-1)
target_volume.columns = ['VC','VI']


table1 = pd.merge(acquire_count,target_count,left_index=True,right_index=True,suffixes=('_acquire','_target'))
table1['N_acquire'] = table1['NI_acquire']+table1['NC_acquire']
table1['N_target'] = table1['NI_target']+table1['NC_target']
table1['%NC_acquire'] = table1['NC_acquire']/table1['NI_acquire'] 
table1['%NC_target'] = table1['NC_target']/table1['NI_target']


table11 = pd.merge(acquire_volume,target_volume,left_index=True,right_index=True,suffixes=('_acquire','_target'))
table1 =pd.merge(table1,table11,left_index=True,right_index=True)
table1.drop(columns=['NI_acquire','NI_target','VI_acquire','VI_target'],inplace=True)
table1.to_csv('./table_out/Table1.csv')

##### Construct x - open B-A

In [4]:
policy = pd.read_excel('../data/policy.xlsx')
open_policy = policy.iloc[:,2:]
open_policy['state'] = open_policy['State.1'].apply(lambda x:state_full[x])
years = range(1980,1998)
years = np.tile(years, len(open_policy))

states = open_policy['state'].values
shuffle_states = np.tile(states,len(states))
shuffle_years = np.tile(years,len(states))
states = states.repeat(18)

open_df = pd.DataFrame([states,years]).T
open_df.columns = ['state','year']
open_df = pd.merge(open_df,open_policy,left_on = 'state',right_on='state')
open_df['open'] = open_df.apply(lambda x: 1 if x['year']>=x['Interstate banking'] else 0,axis =1)

open_diff = pd.merge(open_df,open_df,how = 'outer',left_on = 'year',right_on = 'year',suffixes=('_target','_acquire'))
open_diff = open_diff[open_diff['state_target']!=open_diff['state_acquire']]
open_diff['open_B-A'] = open_diff['open_target']-open_diff['open_acquire']
open_diff.rename(columns={'state_target':'Target State','state_acquire':'Acquiror State','year':'announce_year'},inplace=True)
open_diff['announce_year'] = open_diff['announce_year'].astype(int)

##### Construct y - acquire volume B-A

In [5]:
group_acq_tar_year =ma_df.groupby(by = ['Acquiror State','Target State','announce_year'])
group_acq_year = ma_df.groupby(by = ['Acquiror State','announce_year'])

count_df = group_acq_tar_year['Target Name'].count()
count_df = count_df.reset_index()
count_df = count_df[count_df['Target State']!=count_df['Acquiror State']]
count_all =  group_acq_year['Target Name'].count()
count_all = count_all.reset_index()
count_df = pd.merge(count_df,count_all,left_on=['Acquiror State','announce_year'],right_on=['Acquiror State','announce_year'],suffixes=('_acquire','_all'))
count_df['Acq_Count_A-B'] = count_df['Target Name_acquire']/count_df['Target Name_all']
count_df['announce_year'] = count_df['announce_year'].astype(int)

##NOTE: Can we assume those with no value as 0???
volume_df = group_acq_tar_year['Value of\nTransaction\n($mil)'].agg(lambda x: x.sum(skipna= False))#sum()
volume_df = volume_df.reset_index()
volume_df = volume_df[volume_df['Target State']!=volume_df['Acquiror State']]
volume_all =  group_acq_year['Value of\nTransaction\n($mil)'].agg(lambda x: x.sum(skipna= False))# sum()
volume_all = volume_all.reset_index()
volume_df['announce_year'] = volume_df['announce_year'].astype(int)
volume_df = pd.merge(volume_df,volume_all,left_on=['Acquiror State','announce_year'],right_on=['Acquiror State','announce_year'],suffixes=('_acquire','_all'))
volume_df['Acq_Vol_A-B'] = volume_df['Value of\nTransaction\n($mil)_acquire']/volume_df['Value of\nTransaction\n($mil)_all']
volume_df.loc[volume_df['Acq_Vol_A-B'].isna(),'Acq_Vol_A-B'] = 'unknown'

acq_df = pd.merge(count_df,volume_df,on = ['Acquiror State','Target State','announce_year'])
acq_df['announce_year'] = acq_df['announce_year'].astype(int)

reg_df = pd.merge(acq_df,open_diff,how = 'right',on = ['Acquiror State','Target State','announce_year'])
reg_df.loc[reg_df['Acq_Count_A-B'].isna(),'Acq_Count_A-B'] = 0
reg_df.loc[reg_df['Acq_Vol_A-B'].isna(),'Acq_Vol_A-B'] = 0
reg_df.loc[reg_df['Acq_Vol_A-B']=='unknown','Acq_Vol_A-B'] = None


##### Construct controls

In [6]:
## within state 
volume_all = pd.merge(open_df[['state', 'year']],volume_all,how='left',left_on=['state', 'year'],right_on=['Acquiror State','announce_year'])
volume_all['Value of\nTransaction\n($mil)'] = volume_all['Value of\nTransaction\n($mil)'].fillna(0)

volume_all.sort_values(by=['state','year'],ascending=[True,True],inplace=True)
volume_all['shift_Value of\nTransaction\n($mil)'] = volume_all.groupby(by = ['state'])['Value of\nTransaction\n($mil)'].shift()
def get_growth(x):
    if x['shift_Value of\nTransaction\n($mil)']!=0:
        return (x['Value of\nTransaction\n($mil)']-x['shift_Value of\nTransaction\n($mil)'])/x['shift_Value of\nTransaction\n($mil)'] 
    elif x['Value of\nTransaction\n($mil)']==0:
        return 0
    else:
        return 1

volume_all['growth'] = volume_all.apply(lambda x:get_growth(x),axis=1)
reg_df = pd.merge(reg_df,volume_all[['state', 'year','growth']],left_on=['Acquiror State','announce_year'],right_on=['state','year'])
reg_df = pd.merge(reg_df,volume_all[['state', 'year','growth']],left_on=['Target State','announce_year'],right_on=['state','year'],suffixes=('_acquire','_target'))
##

In [7]:
## GDP growth and GDP percapita
gdp = pd.read_csv('../data/GDP-by-state-industry.csv')
gdp.drop(columns=['ID ','COMPONENT                                                    ','FIPS   '],inplace=True)
gdp = gdp.iloc[:-6]
gdp.columns = [c.strip() for c in gdp.columns]
##NOTE: There are some cases that it is labeled as (L), which means the value is below 0.5 million, we simply replace it with 0.5
##NOTE: we do not know whether or not it is real or nominal GDP
gdp = gdp.replace('   (L)  ','0.5')
gdp['REGION'] = gdp['REGION'].str.strip()
gdp.loc[gdp['REGION']=='Dist. of Col.','REGION'] = 'D. of Columbia'

gdp_state = gdp[gdp['CODE']==0]
gdp_state = gdp_state[:-9]
gdp_state.drop(columns=[ 'CODE','INDUSTRY'],inplace=True)
states = gdp_state['REGION'].values
states = states.repeat(21)

t = gdp_state.iloc[:,1:].stack()
t = t.reset_index()
t.columns = ['index','year','GDP']
t['GDP'] = t['GDP'].astype(float)
t['state'] = states
t.drop(columns=['index'],inplace=True)
t['GDP_shift'] = t.groupby('state')['GDP'].shift()
t['GDP_growth'] = (t['GDP']-t['GDP_shift'])/t['GDP_shift']

t['year'] = t['year'].astype(int)


population = pd.read_excel('../data/popest-annual-historical.xls',sheet_name='States')
population.loc[population['Area Name']=='District of Columbia','Area Name'] = 'D. of Columbia'
population.columns =[str(c).strip() for c in population.columns]
population.rename(columns={'Area Name':'REGION'},inplace=True)

tt = population.iloc[:,1:].stack()
tt = tt.reset_index()
tt.columns = ['index','year','population']
tt['population'] = tt['population'].astype(int)
states_po= population['REGION'].values
states_po = states_po.repeat(41)
tt['state'] = states_po
tt.drop(columns=['index'],inplace=True)
tt['year'] = tt['year'].astype(int)

tem = pd.merge(t,tt,on = ['state','year'])
tem['GDP_per_capita'] = tem['GDP']/tem['population']

reg_df['announce_year'] = reg_df['announce_year'].astype(int)
tt = pd.merge(reg_df,tem,left_on=['Acquiror State','announce_year'],right_on=['state','year'])
reg_df = pd.merge(tt,tem,left_on=['Target State','announce_year'],right_on=['state','year'],suffixes=('_acquire','_target'))
reg_df.drop(columns=['state_acquire','GDP_shift_acquire','year_target','state_target','GDP_shift_target','year_acquire'],inplace=True)
reg_df['GDP_growth_B-A'] = reg_df['GDP_growth_target'] - reg_df['GDP_growth_acquire']
reg_df['GDP_per_capita_B-A'] = reg_df['GDP_per_capita_target'] - reg_df['GDP_per_capita_acquire']
reg_df

C:\Users\chuxu\AppData\Local\Temp\ipykernel_5136\2919322394.py:50: FutureWarning: Passing 'suffixes' which cause duplicate columns {'state_acquire', 'year_acquire'} in the result is deprecated and will raise a MergeError in a future version.
  reg_df = pd.merge(tt,tem,left_on=['Target State','announce_year'],right_on=['state','year'],suffixes=('_acquire','_target'))


,Acquiror State,Target State,announce_year,Target Name_acquire,Target Name_all,Acq_Count_A-B,Value of\nTransaction\n($mil)_acquire,Value of\nTransaction\n($mil)_all,Acq_Vol_A-B,State.1_target,Interstate banking_target,Intrastate branching_target,open_target,State.1_acquire,Interstate banking_acquire,Intrastate branching_acquire,open_acquire,open_B-A,growth_acquire,growth_target,GDP_acquire,GDP_growth_acquire,population_acquire,GDP_per_capita_acquire,GDP_target,GDP_growth_target,population_target,GDP_per_capita_target,GDP_growth_B-A,GDP_per_capita_B-A
0,Alabama,Alaska,1980,NaN,NaN,0.000000,NaN,NaN,0,AK,1982,before 1970,0,AL,1987,1981,0,0,NaN,NaN,36006.0,0.073684,3900368,0.009231,15138.0,0.393538,405315,0.037349,0.319853,0.028117
1,Arkansas,Alaska,1980,NaN,NaN,0.000000,NaN,NaN,0,AK,1982,before 1970,0,AR,1989,1994,0,0,NaN,NaN,20100.0,0.061134,2288738,0.008782,15138.0,0.393538,405315,0.037349,0.332404,0.028567
2,Arizona,Alaska,1980,NaN,NaN,0.000000,NaN,NaN,0,AK,1982,before 1970,0,AZ,1986,before 1970,0,0,NaN,NaN,30431.0,0.114444,2737774,0.011115,15138.0,0.393538,405315,0.037349,0.279094,0.026233
3,California,Alaska,1980,NaN,NaN,0.000000,NaN,NaN,0,AK,1982,before 1970,0,CA,1987,before 1970,0,0,NaN,NaN,324407.0,0.111394,23800800,0.013630,15138.0,0.393538,405315,0.037349,0.282144,0.023719
4,Colorado,Alaska,1980,NaN,NaN,0.000000,NaN,NaN,0,AK,1982,before 1970,0,CO,1988,1991,0,0,NaN,NaN,38151.0,0.132212,2908803,0.013116,15138.0,0.393538,405315,0.037349,0.261326,0.024233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45895,Washington,Alabama,1997,NaN,NaN,0.000000,NaN,NaN,0,AL,1987,1981,1,WA,1987,1985,1,0,0.0,0.0,174412.0,0.078215,5674747,0.030735,102533.0,0.046885,4367935,0.023474,-0.031329,-0.007261
45896,Wisconsin,Alabama,1997,2.0,95.0,0.021053,43.5,NaN,None,AL,1987,1981,1,WI,1987,1990,1,0,0.0,0.0,148831.0,0.049917,5266213,0.028261,102533.0,0.046885,4367935,0.023474,-0.003032,-0.004787
45897,West Virginia,Alabama,1997,NaN,NaN,0.000000,NaN,NaN,0,AL,1987,1981,1,WV,1988,1987,1,0,0.0,0.0,38517.0,0.031355,1819113,0.021174,102533.0,0.046885,4367935,0.023474,0.015530,0.002301
45898,Wyoming,Alabama,1997,NaN,NaN,0.000000,NaN,NaN,0,AL,1987,1981,1,WY,1987,1988,1,0,0.0,0.0,16041.0,0.019641,489451,0.032773,102533.0,0.046885,4367935,0.023474,0.027244,-0.009299


In [8]:
# Industry dissimilarity A&B
gdp = gdp[~gdp['CODE'].isin([0,5000,10000,30000,40000,50000,60000,70000,80000,90000,100000,110000])]
industry_df = pd.DataFrame()
states = gdp.REGION.unique()
for r in states:
    for r1 in states:
        if r==r1:
            continue
        curr_r = gdp[gdp['REGION']==r]
        curr_r1 = gdp[gdp['REGION']==r1]

        curr_r_ind = curr_r['CODE']
        curr_r_ind = np.tile(curr_r_ind,21)
        curr_r_t = curr_r.iloc[:,3:].stack()
        curr_r_t = curr_r_t.reset_index()
        curr_r_t['CODE'] = curr_r_ind
        curr_r_t.columns = ['index','year','GDP','CODE']
        curr_r_t.drop(columns=['index'],inplace=True)

        curr_r1_ind = curr_r1['CODE']
        curr_r1_ind = np.tile(curr_r1_ind,21)
        curr_r1_t = curr_r1.iloc[:,3:].stack()
        curr_r1_t = curr_r1_t.reset_index()
        curr_r1_t['CODE'] = curr_r1_ind
        curr_r1_t.columns = ['index','year','GDP','CODE']
        curr_r1_t.drop(columns=['index'],inplace=True)

        curr_r_t['GDP'] = curr_r_t['GDP'].apply(lambda x: float(x.strip()))
        curr_r1_t['GDP'] = curr_r1_t['GDP'].apply(lambda x: float(x.strip()))
        curr_r_GDP_sum = curr_r_t.groupby(by = 'year')['GDP'].sum()
        curr_r1_GDP_sum = curr_r1_t.groupby(by = 'year')['GDP'].sum()
        
        curr_r_t = pd.merge(curr_r_t,curr_r_GDP_sum,left_on='year',right_index=True,suffixes=('','_tot'))
        curr_r_t['share'] = curr_r_t['GDP']/curr_r_t['GDP_tot']
        curr_r1_t = pd.merge(curr_r1_t,curr_r1_GDP_sum,left_on='year',right_index=True,suffixes=('','_tot'))
        curr_r1_t['share'] = curr_r1_t['GDP']/curr_r1_t['GDP_tot']

        curr_t = pd.merge(curr_r_t,curr_r1_t,how='outer',on = ['year','CODE'])
        curr_t[['share_x','share_y']] = curr_t[['share_x','share_y']].fillna(0.0)
        curr_t['GDP_diff'] = curr_t.apply(lambda row: (row['share_x']-row['share_y'])**2,axis=1)

        curr_t = curr_t.groupby('year')['GDP_diff'].sum()
        curr_t = curr_t.apply(lambda x:np.sqrt(x))

        curr_t = curr_t.reset_index()
        curr_t['Acquiror State']=r
        curr_t['Target State'] = r1

        industry_df = pd.concat([industry_df,curr_t])
        print('Finished ',r,' and ',r1)
print('END!!!')

industry_df = industry_df[~industry_df['Acquiror State'].isin(['UNITED STATES',
       'NEW ENGLAND', 'MIDEAST', 'GREAT LAKES', 'PLAINS', 'SOUTHEAST',
       'SOUTHWEST', 'ROCKY MTN', 'FAR WEST'])]
industry_df = industry_df[~industry_df['Target State'].isin(['UNITED STATES',
       'NEW ENGLAND', 'MIDEAST', 'GREAT LAKES', 'PLAINS', 'SOUTHEAST',
       'SOUTHWEST', 'ROCKY MTN', 'FAR WEST'])]
industry_df['year'] = industry_df['year'].astype(int)
industry_df.rename(columns={'GDP_diff':'Industry_dissimilarity_B-A'},inplace=True)
reg_df = pd.merge(reg_df,industry_df, left_on=['Acquiror State','Target State','announce_year'],right_on = ['Acquiror State','Target State','year'])

Finished  Alabama  and  Alaska
Finished  Alabama  and  Arizona
Finished  Alabama  and  Arkansas
Finished  Alabama  and  California
Finished  Alabama  and  Colorado
Finished  Alabama  and  Connecticut
Finished  Alabama  and  Delaware
Finished  Alabama  and  D. of Columbia
Finished  Alabama  and  Florida
Finished  Alabama  and  Georgia
Finished  Alabama  and  Hawaii
Finished  Alabama  and  Idaho
Finished  Alabama  and  Illinois
Finished  Alabama  and  Indiana
Finished  Alabama  and  Iowa
Finished  Alabama  and  Kansas
Finished  Alabama  and  Kentucky
Finished  Alabama  and  Louisiana
Finished  Alabama  and  Maine
Finished  Alabama  and  Maryland
Finished  Alabama  and  Massachusetts
Finished  Alabama  and  Michigan
Finished  Alabama  and  Minnesota
Finished  Alabama  and  Mississippi
Finished  Alabama  and  Missouri
Finished  Alabama  and  Montana
Finished  Alabama  and  Nebraska
Finished  Alabama  and  Nevada
Finished  Alabama  and  New Hampshire
Finished  Alabama  and  New Jersey
Finis

In [9]:
# Economic correlation A&B 
from datetime import datetime
coincide = pd.read_excel('../data/coincident-index.xls')
#coincide = coincide[coincide['month']==12]
coincide.set_index('Date',inplace=True)
#coincide.drop(columns=['month','Date'],inplace=True)
coincide_df  = coincide.iloc[:,1:-3].stack()
coincide_df = coincide_df.reset_index()
coincide_df.columns=['index','level_1','coincident_index']
#coincide = coincide[coincide['level_1']!='US']
coincide_df['state'] = coincide_df['level_1'].apply(lambda x:state_full[x])
coincide_df.drop(columns=['level_1'],inplace=True)
coincide = coincide[coincide['year']<1998]

coincide_df['year'] = coincide_df['index'].dt.year
#coincide_df['month'] = coincide_df['Date'].dt.month
coincide = coincide_df[coincide_df['year']<1998]

def get_correlation(x):
    first_series = coincide[(coincide['state']==x['Acquiror State'])&(coincide['year']==x['announce_year'])]
    second_series = coincide[(coincide['state']==x['Target State'])&(coincide['year']==x['announce_year'])]
    t = pd.merge(first_series,second_series,on='index')
    return t[['coincident_index_x','coincident_index_y']].corr().values[0,1]

reg_df['Economic_correlation_AB'] = reg_df.apply(lambda row: get_correlation(row),axis=1)
reg_df

KeyError: 'year'

In [ ]:
## Tobins'q B-A
compustat = pd.read_csv('../data/compustats.csv')
compustat = compustat[compustat['loc']=='USA']
compustat = compustat[['fyear','cusip','conm','state','at','prcc_f','csho','ceq','txdb']]
compustat = compustat[compustat['at']>0.1]
compustat['tobinq'] = (compustat['prcc_f']*compustat['csho']+compustat['at']-compustat['ceq']-compustat['txdb'])/compustat['at']
tobinq_df = compustat.groupby(by = ['state','fyear'])['tobinq'].mean()
tobinq_df =tobinq_df.reset_index()
tobinq_df = tobinq_df[~tobinq_df['state'].isin(['PR','VI'])]
tobinq_df['state_full'] = tobinq_df['state'].apply(lambda x: state_full[x])

reg_df = pd.merge(reg_df,tobinq_df,left_on=['Acquiror State','announce_year'],right_on=['state_full','fyear'])
reg_df = pd.merge(reg_df,tobinq_df,left_on=['Target State','announce_year'],right_on=['state_full','fyear'],suffixes=('_acquire','_target'))
reg_df['tobinq_B-A'] = reg_df['tobinq_target']-reg_df['tobinq_acquire']

C:\Users\chuxu\AppData\Local\Temp\ipykernel_12928\1570192035.py:2: DtypeWarning: Columns (30,35,948,949,950,955,969,970,980) have mixed types. Specify dtype option on import or set low_memory=False.
  compustat = pd.read_csv('../data/compustats.csv')


In [ ]:
# stock return B-A
return_df = pd.read_csv('../data/stock_return.csv')
return_df = return_df[['CUSIP','CUSIP9','YYYYMM','MthRet']]
return_df['CUSIP9'] = return_df['CUSIP9'].astype(str)
compustat_state = compustat[['cusip','state']]
compustat_state['cusip'] = compustat_state['cusip'].astype(str)
compustat_state.drop_duplicates(inplace=True)
return_df = pd.merge(return_df,compustat_state,left_on='CUSIP9',right_on='cusip')

return_df['year'] = return_df['YYYYMM']//100
return_df['payoff'] = return_df['MthRet']+1
return_df.sort_values(by = ['CUSIP9','YYYYMM'],ascending=[True,True],inplace=True)

ret_df = return_df.groupby(by = ['CUSIP9','year'])['payoff'].prod()-1
ret_df = ret_df.reset_index()
ret_df = pd.merge(ret_df,compustat_state,left_on='CUSIP9',right_on='cusip')
ret_df = ret_df.groupby(by=['state','year'])['payoff'].mean()
ret_df = ret_df.reset_index()
ret_df = ret_df[~ret_df['state'].isin(['PR','VI'])]
ret_df['state_full'] = ret_df['state'].apply(lambda x: state_full[x])

reg_df = pd.merge(reg_df,ret_df,how = 'left',left_on=['Acquiror State','announce_year'],right_on=['state_full','year'])
reg_df = pd.merge(reg_df,ret_df,how = 'left',left_on=['Target State','announce_year'],right_on=['state_full','year'],suffixes=('_acquire','_target'))
reg_df['stock_return_B-A'] = reg_df['payoff_target'] - reg_df['payoff_acquire']

reg_df.drop(columns=['year_x','year_y', 'State.1_acquire','State.1_target','fyear_target','fyear_acquire','year','state_full_target','state_full_acquire','state_target','state_acquire'],inplace=True)

C:\Users\chuxu\AppData\Local\Temp\ipykernel_12928\2567774867.py:2: DtypeWarning: Columns (6,7,8,9,15) have mixed types. Specify dtype option on import or set low_memory=False.
  return_df = pd.read_csv('../data/stock_return.csv')
C:\Users\chuxu\AppData\Local\Temp\ipykernel_12928\2567774867.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compustat_state['cusip'] = compustat_state['cusip'].astype(str)
C:\Users\chuxu\AppData\Local\Temp\ipykernel_12928\2567774867.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compustat_state.drop_duplicates(inplace=True)
C:\

In [ ]:
## unemployment B-A
unemployment = pd.read_excel('../data/emp-unemployment.xls',header=5,sheet_name='States')
unemployment = unemployment.iloc[:-9,:]
unemployment.drop(columns=['Fips'],inplace=True)
unemployment.set_index('Area',inplace=True)
unemployment = unemployment.stack()
unemployment = unemployment.reset_index()
unemployment.loc[unemployment['Area']=='District of Columbia','Area']='D. of Columbia'
unemployment = unemployment[unemployment['Area']!='United States']
unemployment.columns = ['Area','year','unemployment']

reg_df = pd.merge(reg_df,unemployment,left_on=['Acquiror State','announce_year'],right_on=['Area','year'])
reg_df = pd.merge(reg_df,unemployment,left_on=['Target State','announce_year'],right_on=['Area','year'],suffixes=('_acquire','_target'))
reg_df['unemployment_B-A'] = reg_df['unemployment_target']- reg_df['unemployment_acquire']

In [ ]:
## Anti-combination B-A
def get_anti(x,target = 'target'):
    if str(x['Year BC Passed_%s'%target])=='nan':
        return 0
    else:
        if x['Year BC Passed_%s'%target]> x['announce_year']:
            return 0
        else:
            return 1
anti = pd.read_excel('../data/policy.xlsx')
anti = anti[['State','Year BC Passed']]
anti.dropna(how='all',inplace=True)

reg_df = pd.merge(reg_df,anti,how='left',left_on=['Acquiror State'],right_on='State')
reg_df = pd.merge(reg_df,anti,how='left',left_on=['Target State'],right_on='State',suffixes=('_acquire','_target'))

reg_df['anti_competition_acquire'] = reg_df.apply(lambda x: get_anti(x,'acquire'),axis=1)
reg_df['anti_competition_target'] = reg_df.apply(lambda x: get_anti(x,'target'),axis=1)
reg_df['anti_competition_B-A'] = reg_df['anti_competition_target'] - reg_df['anti_competition_acquire']
reg_df.drop(columns=['State_acquire','State_target'],inplace=True)

In [ ]:
## corporate tax rate B-A
tax_df = pd.read_excel('../data/State Corporate Income Tax Rates, 2000-2014.xlsx',sheet_name='arrange')
reg_df = pd.merge(reg_df,tax_df,left_on='Acquiror State',right_on='State')
reg_df = pd.merge(reg_df,tax_df,left_on='Target State',right_on='State',suffixes=('_acquire','_target'))
reg_df['Corporate_income_tax_B-A'] = reg_df['rate_target'] - reg_df['rate_acquire']
reg_df.to_excel('baseline.xlsx')